In [44]:
import pandas as pd
import numpy as np
import math
from math import log
import pprint

In [2]:
dataset = pd.read_csv('weather.csv')
df = pd.read_csv('weather.csv')
salida = "play"

In [10]:
dataset.head(13)

,outlook,temperature,humidity,windy,play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes
5,rainy,cool,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,normal,False,yes
8,rainy,mild,high,True,no
9,sunny,hot,high,False,no


In [11]:
#columnas

columnas = []
for i in dataset:
    columnas.append(i)
    print(columnas)
columnas
print(columnas)

['outlook']
['outlook', 'temperature']
['outlook', 'temperature', 'humidity']
['outlook', 'temperature', 'humidity', 'windy']
['outlook', 'temperature', 'humidity', 'windy', 'play']
['outlook', 'temperature', 'humidity', 'windy', 'play']


In [50]:
# entropia de PLAY

eps = np.finfo(float).eps


dataset = pd.read_csv("weather.csv")
entropia_play = 0

valores = dataset.play.unique()
print(valores)

for valor in valores:
    print(dataset.play.value_counts()[valor])
    
    fraccion = dataset.play.value_counts()[valor]/len(dataset.play)  
    entropia_play += -fraccion*np.log2(fraccion)

print("Entropia Play:", entropia_play)

['yes' 'no']
9
5
Entropia Play: 0.9402859586706309


In [52]:
# Entropia para los demas atributos

def entropia(dataset,atributos):
    clases_variable = dataset.play.unique()
    variables = dataset[atributos].unique()


    entropia_atributo = 0
    for variable in variables:
        entropia_caracteristica = 0
        for target_variable in clases_variable:
            num = len(dataset[atributos][dataset[atributos]==variable][dataset.play ==target_variable])

            den = len(dataset[atributos][dataset[atributos]==variable])
            fraccion = num/(den+eps)
            entropia_caracteristica += -fraccion*log(fraccion+eps)
        fraccion2 = den/len(dataset)
        entropia_atributo += -fraccion2*entropia_caracteristica

    return(abs(entropia_atributo))

In [54]:
entropia_atributos = {k:entropia(dataset,k) for k in dataset.keys()[:-1]}
entropia_atributos

print(entropia_atributos)

{'outlook': 0.48072261929232574, 'temperature': 0.6315010221774202, 'humidity': 0.54651221149444, 'windy': 0.618397445736438}


In [55]:
#Info Gain

def infogain(dataset_entropia,entropia_atributo):
    return(dataset_entropia-entropia_atributo)


In [56]:
IG = {k:infogain(entropia_play,entropia_atributos[k]) for k in entropia_atributos}


In [40]:
print(IG)

{'outlook': 0.4595633393783052, 'temperature': 0.3087849364932107, 'humidity': 0.39377374717619096, 'windy': 0.32188851293419296}


In [57]:
# Entropia nuev

def encontrar_entropia(dataset):
    Class = dataset.keys()[-1] 
    entropy = 0
    valores = dataset[Class].unique()
    for valor in valores:
        fraccion = dataset[Class].value_counts()[valor]/len(dataset[Class])
        entropy += -fraccion*np.log2(fraccion)
    return entropy

In [67]:
# recursividad ->

def encontrar_entropia_atributos(dataset,atributos):
    
    Class = dataset.keys()[-1]   
    clases_variable = dataset[Class].unique()  
    variables = dataset[atributos].unique()
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in clases_variable:
            num = len(dataset[atributos][dataset[atributos]==variable][dataset[Class] ==target_variable])
            den = len(dataset[atributos][dataset[atributos]==variable])
            fraccion = num/(den+eps)
            entropy += -fraccion*log(fraccion+eps)
            print(entropy)
        fraccion2 = den/len(dataset)
        entropy2 += -fraccion2*entropy
    return abs(entropy2)


def encontrar_mejor(dataset):
    Entropy_att = []
    IG = []
    for key in dataset.keys()[:-1]:
        
        IG.append(encontrar_entropia(dataset)-encontrar_entropia_atributos(dataset,key))
    return dataset.keys()[:-1][np.argmax(IG)]
  
def nueva_tabla(dataset, nodo,valor):
    return dataset[dataset[nodo] == valor].reset_index(drop=True)


def crearArbol(dataset,arbol=None): 
    Class = dataset.keys()[-1] 
    
    nodo = encontrar_mejor(dataset)
    
    attValue = np.unique(dataset[nodo])
 
    if arbol is None:                    
        arbol={}
        arbol[nodo] = {}
    
    for valor in attValue:
        
        subtable = nueva_tabla(dataset,nodo,valor)
        clValue,counts = np.unique(subtable['play'],return_counts=True)                        
        
        if len(counts)==1:
            arbol[nodo][valor] = clValue[0]                                                    
        else:        
            arbol[nodo][valor] = crearArbol(subtable)
                   
    return arbol


In [68]:
arbolweather = crearArbol(dataset)

-2.2204460492503128e-16
-2.2204460492503128e-16
0.3064953742595942
0.6730116670092561
0.3665162927496618
0.6730116670092561
0.3465735902799724
0.6931471805599448
0.21576155433883548
0.5623351446188078
0.2703100720721094
0.6365141682948123
0.36312765445165845
0.6829081047004713
0.1321291541376498
0.4101163182884086
0.21576155433883548
0.5623351446188078
0.3465735902799724
0.6931471805599448
0.3064953742595942
0.6730116670092561
0.2703100720721094
0.6365141682948123
0.3465735902799724
0.6931471805599448
0.3465735902799724
0.6931471805599448
0.2703100720721094
0.6365141682948123
-2.2204460492503128e-16
-2.2204460492503128e-16
0.0
-2.2204460492503128e-16
0.3064953742595942
0.6730116670092561
-2.2204460492503128e-16
-2.2204460492503128e-16
0.3465735902799724
0.6931471805599448
0.0
0.0
-2.2204460492503128e-16
-2.2204460492503128e-16
0.0
-2.2204460492503128e-16
0.2703100720721094
0.6365141682948123
0.3465735902799724
0.6931471805599448


In [69]:
print(arbolweather)

{'outlook': {'overcast': 'yes', 'rainy': {'windy': {False: 'yes', True: 'no'}}, 'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}
